In [743]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [744]:
liberal = ["Germany", "Sweden", "France", "Italy"]
conservative = ["Poland", "Bulgaria", "Romania", "Hungary"]
countries = liberal + conservative

def load_data(filename):
    df = pd.read_csv(f"../processed_data/{filename}")
    df = df[df["geo"].isin(countries)]
    df["politics"] = np.where(df["geo"].isin(liberal), "liberal", "conservative")
    return df

files = {
    "migrants": "migrants_all_valid_permits.csv",
    "crimes": "crimes_crime_by_category.csv", 
    "unemployment": "unemployment_by_citizenship.csv",
    "gdp": "HICP_annual_indices_for_all_goods.csv",
    "fertility": "total_fertility_rate.csv",
    "immigration_detail": "immigrants_by_country_groups.csv",
    "ordered_to_leave": "third_country_nationals_ordered_to_leave.csv",
    "illegally_present": "third_country_illegaly_present.csv",
    "level_of_trust": "level_of_trust.csv"
}

data = {name: load_data(filename) for name, filename in files.items()}

politic_colors = {"liberal": "blue", "conservative": "red"}

In [745]:
def plot_trend(df, value_col, title):
    fig = go.Figure()
    plot_data = df[["geo", "year", "politics", value_col]].dropna()
    country_year_data = plot_data.groupby(["geo", "year", "politics"])[value_col].sum().reset_index()
    
    for country, country_data in country_year_data.groupby("geo"):
        politic_group = country_data["politics"].iat[0]
        fig.add_trace(go.Scatter(
            x=country_data["year"], y=country_data[value_col],
            mode="markers", showlegend=False,
            marker=dict(color=politic_colors[politic_group], opacity=0.6),
            name=country
        ))
    
    mean_data = country_year_data.groupby(["politics", "year"])[value_col].mean().reset_index()
    for politic_group, group_data in mean_data.groupby("politics"):
        fig.add_trace(go.Scatter(
            x=group_data["year"], y=group_data[value_col],
            mode="lines", name=politic_group,
            line=dict(color=politic_colors[politic_group], width=2)
        ))
    
    fig.update_layout(title=title, xaxis_title="Rok", yaxis_title="Wartość")
    fig.show()

plot_trend(data["migrants"], "Migrants number per 100k inhabitants", "Trend migracji")
plot_trend(data["crimes"], "Crimes num", "Trend przestępczości") 
plot_trend(data["gdp"], "HICP value", "Trend PKB")
plot_trend(data["fertility"], "Total fertility rate", "Trend dzietności")

In [746]:
df_unemp = data["unemployment"][["geo", "year", "politics", "citizen", "OBS_VALUE"]].dropna()
unemp_agg = df_unemp.groupby(["politics", "citizen", "year"])["OBS_VALUE"].mean().reset_index()

unemp_filter = unemp_agg[
    (unemp_agg["politics"] == "liberal") |
    ((unemp_agg["politics"] == "conservative") & (unemp_agg["citizen"] == "Reporting country"))
]

fig = px.line(
    unemp_filter, x="year", y="OBS_VALUE",
    color="politics", line_dash="citizen",
    title="Stopa bezrobocia w krajach liberalnych i konserwatywnych",
)
fig.update_layout(yaxis_title="Stopa bezrobocia", xaxis_title="Rok")
fig.show()

In [747]:
df_mig = data["migrants"][["geo", "year", "Migrants number per 100k inhabitants", "politics"]]
df_fert = data["fertility"][["geo", "year", "Total fertility rate", "politics"]]

df_scatter = (
    df_mig
    .merge(df_fert, on=["geo", "year", "politics"])
    .dropna(subset=["Migrants number per 100k inhabitants", "Total fertility rate"])
)

agg_pt = df_scatter.groupby(["geo", "politics"], as_index=False).agg(
    migrants_sum=("Migrants number per 100k inhabitants", "sum"),
    fertility_sum=("Total fertility rate", "sum")
)

fig = px.scatter(
    agg_pt,
    x="migrants_sum", y="fertility_sum",
    color="politics", text="geo",
    trendline="ols",
    title="Wskaźnik migracji a wskaźnik dzietności w krajach liberalnych i konserwatywnych"
)
fig.update_traces(textposition="top center")
fig.update_layout(xaxis_title="Wskaźnik migracji", yaxis_title="Wskaźnik dzietności")
fig.show()

**Wnioski:**
- Wyższa imigracja → niższa dzietność
- Kraje konserwatywne: słabszy wpływ imigracji na dzietność (R² ≈ 0.87)
- Kraje liberalne: silniejszy wpływ imigracji na dzietność (R² ≈ 0.98)

In [748]:
def plot_boxplot(df, value_col, title, ylabel):
    df_agg = df.groupby(["geo", "politics", "year"], as_index=False)[value_col].sum()
    fig = px.box(
        df_agg,
        x="politics",
        y=value_col,
        color="politics",
        title=title,
        labels={value_col: ylabel, "politics": "Grupa polityczna"},
    )
    fig.show()

plot_boxplot(data["migrants"], "Migrants number per 100k inhabitants", 
            "Rozkład liczby migrantów na kraj wg grup politycznych", "Roczna liczba migrantów")

plot_boxplot(data["crimes"], "Crimes num", 
            "Rozkład wskaźnika przestępczości na kraj wg grup politycznych", "Roczny wskaźnik przestępczości")

plot_boxplot(data["gdp"], "HICP value", 
            "Rozkład PKB per capita wg grup politycznych", "Roczny PKB per capita")

plot_boxplot(data["fertility"], "Total fertility rate", 
            "Rozkład wskaźnika dzietności na kraj wg grup politycznych", "Roczny wskaźnik dzietności")

In [749]:
df_mig_origin = data["immigration_detail"].copy()

target_aggregated_groups = [ 
    "EU countries except reporting country",
    "Non-EU countries nor reporting country"]


df_mig_origin_filtered = df_mig_origin[df_mig_origin["citizen"].isin(target_aggregated_groups)].copy()


mig_origin_agg = df_mig_origin_filtered.groupby(["year", "politics", "citizen"], as_index=False)["Migrants number"].sum()

fig = px.line(
    mig_origin_agg,
    x="year", y="Migrants number",
    color="politics", line_dash="citizen",
    title="Migracje w krajach liberalnych i konserwatywnych wg grup krajów",
)
fig.update_layout(
    yaxis_title="Liczba migrantów",
    xaxis_title="Rok",
    legend_title_text="Grupa polityczna i kraj pochodzenia"
)
fig.show()


In [751]:
df_ordered_to_leave = data["ordered_to_leave"][["geo", "year", "politics", "Third Country Nationals Ordered to Leave"]].copy()
df_illegally_present = data["illegally_present"][["geo", "year", "politics", "Illegal Third Country Nationals"]].copy()


df_combined_country_level = pd.merge(df_ordered_to_leave, df_illegally_present, on=["geo", "year", "politics"])

df_yearly_agg = df_combined_country_level.groupby(["year", "politics"], as_index=False)[["Third Country Nationals Ordered to Leave", "Illegal Third Country Nationals"]].sum()

fig = px.scatter(
df_yearly_agg,
x="Illegal Third Country Nationals",
y="Third Country Nationals Ordered to Leave",
color="politics",
symbol="politics",
text="year",
title="Nielegalnie przebywający vs. Nakazy opuszczenia",
labels={
    "Illegal Third Country Nationals": "Suma osób przebywających nielegalnie",
    "Third Country Nationals Ordered to Leave": "Suma osób z nakazem opuszczenia kraju",
    "politics": "Grupa polityczna",
    "year": "Rok"
},
color_discrete_map=politic_colors,
)
fig.update_traces(textposition='top center')
fig.show()
